# Single Call Protocol

In an effort to make it as easy as possible to create custom chains, we've implemented a "Runnable" protocol that most components implement. This is a standard interface with a few different methods, which makes it easy to define custom chains as well as making it possible to invoke them in a standard way. The standard interface exposed includes:

- `stream`: stream back chunks of the response
- `invoke`: call the chain on an input
- `batch`: call the chain on a list of inputs

These also have corresponding async methods:

- `astream`: stream back chunks of the response async
- `ainvoke`: call the chain on an input async
- `abatch`: call the chain on a list of inputs async

The type of the input varies by component - eg for a prompt it is a dictionary, for a retriever it is a single string.

Let's take a look at a few different ways to do things!

## PromptTemplate + LLM

A PromptTemplate -> LLM is a core chain that is used in most other larger chains/systems.

In [1]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

/Users/harrisonchase/.pyenv/versions/3.9.1/envs/langchain/lib/python3.9/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.6.13) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [2]:
model = ChatOpenAI()

In [3]:
prompt = ChatPromptTemplate.from_template("tell me a joke about {foo}")

In [4]:
chain = prompt | model

In [5]:
for s in chain.stream({"foo": "bears"}):
    print(s.content, end="")

Why don't bears wear shoes?

Because they have bear feet!

In [6]:
chain.batch([{"foo": "bears"}])

[AIMessage(content="Why don't bears wear shoes?\n\nBecause they have bear feet!", additional_kwargs={}, example=False)]

In [7]:
chain.invoke({"foo": "bears"})

AIMessage(content="Why don't bears use cell phones?\n\nBecause they can't bear to have their calls dropped!", additional_kwargs={}, example=False)

## PromptTemplate + LLM + OutputParser

We can also add in an output parser to easily trasform the raw LLM/ChatModel output into a more workable format

In [8]:
from langchain.schema.output_parser import NoOpOutputParser

In [9]:
chain = prompt | model | NoOpOutputParser()

Notice that this now returns a string - a much more workable format for downstream tasks

In [10]:
chain.invoke({"foo": "bears"})

"Sure, here's a bear joke for you:\n\nWhy don't bears wear shoes?\n\nBecause they already have bear feet!"

## Passthroughs and itemgetter

Often times when constructing a chain you may want to pass along original input variables to future steps in the chain. How exactly you do this depends on what exactly the input is:

- If the original input was a string, then you likely just want to pass along the string. This can be done with `RunnablePassthrough`. For an example of this, see `LLMChain + Retriever`
- If the original input was a dictionary, then you likely want to pass along specific keys. This can be done with `itemgetter`. For an example of this see `Multiple LLM Chains`

In [11]:
from langchain.schema.runnable import RunnablePassthrough
from operator import itemgetter

## LLMChain + Retriever

Let's now look at adding in a retrieval step, which adds up to a "retrieval-augmented generation" chain

In [12]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.runnable import RunnablePassthrough

In [13]:
# Create the retriever
vectorstore = Chroma.from_texts(["harrison worked at kensho"], embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [14]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [15]:
chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | model | NoOpOutputParser()

In [16]:
chain.invoke("where did harrison work?")

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


'Harrison worked at Kensho.'

In [17]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = {
    "context": itemgetter("question") | retriever, 
    "question": itemgetter("question"), 
    "language": itemgetter("language")
} | prompt | model | NoOpOutputParser()

In [18]:
chain.invoke({"question": "where did harrison work", "language": "italian"})

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


'Harrison ha lavorato a Kensho.'

## Multiple LLM Chains

This can also be used to string together multiple LLMChains

In [19]:
from operator import itemgetter

prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template("what country is the city {city} in? respond in {language}")

chain1 = prompt1 | model | NoOpOutputParser()

chain2 = {"city": chain1, "language": itemgetter("language")} | prompt2 | model | NoOpOutputParser()

chain2.invoke({"person": "obama", "language": "spanish"})

'El país en el que se encuentra la ciudad de Honolulu, en el estado de Hawái, es Estados Unidos.'

In [48]:
# NOTE: no caching here, so it's bad!
prompt1 = ChatPromptTemplate.from_template("generate a random color")
prompt2 = ChatPromptTemplate.from_template("what is a fruit of color: {color}")
prompt3 = ChatPromptTemplate.from_template("what is countries flag that has the color: {color}")
prompt4 = ChatPromptTemplate.from_template("What is the color of {fruit} and {country}")
chain1 = prompt1 | model | NoOpOutputParser()
chain2 = {
    "fruit": {"color": chain1} | prompt2 | model | NoOpOutputParser(),
    "country": {"color": chain1} | prompt3 | model | NoOpOutputParser(),
} | prompt4

In [49]:
chain2.invoke({})

ChatPromptValue(messages=[HumanMessage(content='What is the color of A fruit that closely matches the color #FF7F50 is the persimmon. Persimmons are orange-red in color, and some varieties have a shade similar to #FF7F50. and The flag of the country with the color #9F51D9 is not identifiable as a specific country. Flags are unique to each country and have different colors, symbols, and designs.', additional_kwargs={}, example=False)])

## Arbitrary Functions

You can use arbitrary functions in the pipeline by using the `RunnableLambda`

Note that all inputs to these functions need to be a SINGLE argument. If you have a function that accepts multiple arguments, you should write a wrapper that accepts a single input and unpacks it into multiple argument.

In [20]:
from langchain.schema.runnable import RunnableLambda

def length_function(text):
    return len(text)

def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)

def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])

prompt = ChatPromptTemplate.from_template("what is {a} + {b}")

chain1 = prompt | model

chain = {
    "a": itemgetter("foo") | RunnableLambda(length_function),
    "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")} | RunnableLambda(multiple_length_function)
} | prompt | model

In [21]:
chain.invoke({"foo": "bar", "bar": "gah"})

AIMessage(content='3 + 9 equals 12.', additional_kwargs={}, example=False)

## SQL Database

We can also try to replicate our SQLDatabaseChain using this style.

In [53]:
template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}"""
prompt = ChatPromptTemplate.from_template(template)

In [54]:
from langchain.utilities import SQLDatabase

In [55]:
db = SQLDatabase.from_uri("sqlite:///../../../notebooks/Chinook.db")

In [56]:
def get_schema(_):
    return db.get_table_info()

In [57]:
def run_query(query):
    return db.run(query)

In [58]:
sql_response = {
    "schema": RunnableLambda(get_schema),
    "question": itemgetter("question")
} | prompt | model | NoOpOutputParser() | RunnableLambda(run_query)

In [60]:
sql_response.invoke({"question": "How many employees are there?"})

'[(8,)]'

In [51]:
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_template(template)

In [ ]:
{
    "schema": RunnableLambda(get_schema),
    "question": itemgetter("question"),
    
}